In [ ]:
# ============================================
# FRAUD DETECTION SYSTEM WITH CONDITIONAL LOGIC
# ============================================
# This notebook demonstrates advanced CrewAI features:
# - Conditional tasks that execute based on previous results
# - Structured output using Pydantic models
# - Multi-agent collaboration with manager oversight
# - Dynamic task generation based on runtime data

from typing import List
from crewai import Agent, Crew, Task
from pydantic import BaseModel  # For structured output validation
from crewai.tasks.conditional_task import ConditionalTask  # For conditional execution
from crewai.tasks.task_output import TaskOutput  # To access task results
import os 

# Set up API key - Replace with your actual OpenAI API key
os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [ ]:
# ============================================
# CONDITIONAL LOGIC FUNCTION
# ============================================
# This function determines if escalation is needed based on task output
# It will be used to trigger the escalation task conditionally

def is_escalation_needed(output: TaskOutput) -> bool:
    """
    Check if the number of detected anomalies exceeds the threshold.
    
    Parameters:
    - output: The output from the fraud detection task
    
    Returns:
    - True if anomalies > 5 (triggers escalation)
    - False otherwise (skips escalation)
    """
    # Access the structured Pydantic output and count anomalies
    return len(output.pydantic.anomalies) > 5

In [ ]:
# ============================================
# STRUCTURED OUTPUT MODEL
# ============================================
# Define the expected output structure using Pydantic
# This ensures the agent returns data in a consistent, parseable format

class FraudDetectionTaskOutput(BaseModel):
    """
    Defines the structure of fraud detection results.
    
    Benefits of using Pydantic:
    - Type validation
    - Automatic parsing
    - Easy access to structured data
    """
    anomalies: List[str]  # List of transaction IDs flagged as anomalies

### Agents

In [ ]:
# ============================================
# AGENT DEFINITIONS
# ============================================
# Create specialized agents for different aspects of fraud detection

# AGENT 1: Initial Analysis
fraud_analyst = Agent(
    role="Fraud Analyst",
    goal="Analyze transactions to detect anomalies",
    backstory="Experienced in financial fraud detection.",
    # This agent reviews all transactions and identifies suspicious patterns
)

# AGENT 2: Human Review Simulation
escalation_specialist = Agent(
    role="Escalation Specialist",
    goal="Manually review flagged transactions",
    backstory="Handles complex fraud cases requiring human oversight.",
    # This agent only activates when many anomalies are detected (conditional task)
)

# AGENT 3: Report Generation
report_generator = Agent(
    role="Report Generator",
    goal="Prepare a summary fraud detection report",
    backstory="Compiles findings into actionable insights.",
    # This agent creates the final comprehensive report
)

### Tasks

In [ ]:
# ============================================
# TASK DEFINITIONS
# ============================================
# Define the workflow tasks with conditional logic

# DYNAMIC TASK GENERATOR
def create_transaction_tasks(transactions: List[str]):
    """
    Creates fraud analysis tasks dynamically based on transaction list.
    
    This pattern allows you to adapt tasks to different datasets at runtime.
    """
    task1 = Task(
        description=f"Analyze the following transactions for anomalies: {transactions}",
        agent=fraud_analyst,
        expected_output=f"Number of anomalies detected for each transaction.",
        
        # IMPORTANT: Specify output structure for conditional logic
        # This allows subsequent tasks to programmatically access results
        output_pydantic=FraudDetectionTaskOutput,
    )
    return [task1]

# CONDITIONAL TASK: Executes only if condition is met
conditional_task = ConditionalTask(
    description="Escalate flagged transactions if more than 5 anomalies are detected.",
    agent=escalation_specialist,
    
    # KEY FEATURE: This task only runs if is_escalation_needed() returns True
    condition=is_escalation_needed,
    
    expected_output="Validated flagged transactions after manual review."
)

# FINAL TASK: Always executes
task3 = Task(
    description="Generate a summary report on fraud detection results.",
    agent=report_generator,
    expected_output="A detailed fraud detection report."
)

### Manager

In [ ]:
# ============================================
# MANAGER AGENT
# ============================================
# The manager oversees the entire workflow and coordinates between agents

manager_agent = Agent(
    role="Fraud Detection Manager",
    goal="Oversee the fraud detection workflow",
    backstory="Ensures efficient fraud analysis and resolution.",
    # The manager will plan and delegate tasks to specialized agents
)

### List of transactions

In [ ]:
# ============================================
# SAMPLE TRANSACTION DATA
# ============================================
# Realistic transaction dataset with both normal and suspicious transactions

transaction_list = [
    # ========== NORMAL TRANSACTIONS (10) ==========
    # These represent typical daily banking activities
    "TXN1 | Withdrawal | $500 | ATM | New York | 2024-07-01 14:23 | Card Ending: 1234",
    "TXN2 | Online Purchase | $200 | Electronics Store | San Francisco | 2024-07-02 09:45 | IP: 192.168.0.1",
    "TXN3 | Transfer | $1,000 | Bank Account | Zurich | 2024-07-02 17:10 | Beneficiary: John Doe",
    "TXN4 | POS Purchase | $50 | Grocery Store | Berlin | 2024-07-03 12:30 | Card Ending: 5678",
    "TXN5 | Withdrawal | $400 | ATM | Dubai | 2024-07-04 08:15 | Card Ending: 1234",
    "TXN6 | Online Purchase | $150 | Retail Store | Paris | 2024-07-04 20:50 | IP: 10.0.0.2",
    "TXN7 | Transfer | $2,000 | Bank Account | London | 2024-07-05 11:30 | Beneficiary: Jane Smith",
    "TXN8 | POS Purchase | $30 | Coffee Shop | New York | 2024-07-05 15:00 | Card Ending: 5678",
    "TXN9 | Withdrawal | $200 | ATM | San Francisco | 2024-07-06 10:00 | Card Ending: 1234",
    "TXN10 | Online Purchase | $300 | Bookstore | Berlin | 2024-07-06 13:45 | IP: 10.0.0.5",

    # ========== ANOMALIES (10) ==========
    # These contain red flags that should trigger fraud detection
    "TXN11 | Withdrawal | $9,999 | ATM | Las Vegas | 2024-07-07 03:15 | Card Ending: 9999",  # ⚠️ High Amount, Odd Time
    "TXN12 | Transfer | $50,000 | Offshore Account | Cayman Islands | 2024-07-07 05:00 | Beneficiary: Unknown",  # ⚠️ Offshore Account
    "TXN13 | Online Purchase | $5,500 | Luxury Retailer | Unknown Location | 2024-07-07 23:59 | IP: 255.255.255.0",  # ⚠️ Suspicious IP and Location
    "TXN14 | Withdrawal | $7,000 | ATM | Dubai | 2024-07-08 02:30 | Card Ending: 0000",  # ⚠️ Odd Time, High Amount
    "TXN15 | POS Purchase | $1 | Jewelry Store | New York | 2024-07-08 14:00 | Card Ending: 5678",  # ⚠️ Unusually Low Amount (card testing)
    "TXN16 | Online Purchase | $10,000 | Electronics | Hong Kong | 2024-07-08 22:45 | IP: 192.0.2.1",  # ⚠️ Large Online Purchase
    "TXN17 | Transfer | $25,000 | Bank Account | Zurich | 2024-07-09 16:10 | Beneficiary: Suspicious Entity",  # ⚠️ High Value Transfer
    "TXN18 | Withdrawal | $6,000 | ATM | Remote Location | 2024-07-09 04:45 | Card Ending: 8888",  # ⚠️ Remote Location, Odd Time
    "TXN19 | POS Purchase | $9,000 | Luxury Boutique | Paris | 2024-07-09 19:30 | Card Ending: 1234",  # ⚠️ Unusually High POS Purchase
    "TXN20 | Online Purchase | $8,200 | Unknown Store | Unknown | 2024-07-09 21:15 | IP: 0.0.0.0"  # ⚠️ Untraceable IP, Unknown Store
]

# EXPECTED RESULT: Since we have 10 anomalies, and threshold is 5,
# the conditional escalation task WILL be triggered

### Crew

In [ ]:
# ============================================
# CREW ASSEMBLY
# ============================================
# Combine all components into a coordinated workflow

# Step 1: Generate tasks dynamically based on transaction data
tasks = create_transaction_tasks(transaction_list) + [conditional_task, task3]
# This creates: [fraud_analysis_task, conditional_escalation_task, report_task]

# Step 2: Initialize the Crew
crew = Crew(
    # All three specialized agents
    agents=[fraud_analyst, escalation_specialist, report_generator],
    
    # Task list (including conditional task)
    tasks=tasks,
    
    # HIERARCHICAL MANAGEMENT: Manager coordinates the agents
    manager_agent=manager_agent,
    
    # PLANNING MODE: Manager creates execution plan before starting
    # This enables better task coordination
    planning=True,
    
    # VERBOSE: Show detailed execution logs (useful for learning)
    verbose=True
)

In [ ]:
# ============================================
# EXECUTION
# ============================================
# Start the fraud detection workflow

# Run the Crew - this will:
# 1. Fraud Analyst analyzes all 20 transactions
# 2. Identifies 6+ anomalies (triggering escalation)
# 3. Escalation Specialist reviews flagged transactions (conditional task activates)
# 4. Report Generator creates comprehensive summary
result = crew.kickoff()

print("Fraud Detection Result:", result)

# WORKFLOW INSIGHTS:
# - Notice how the conditional task only ran because anomalies > 5
# - The manager coordinated between agents efficiently
# - Structured output (Pydantic) enabled programmatic decision-making
# - This pattern scales to complex business logic with multiple conditions

 
[2025-01-07 20:03:34][INFO]: Planning the crew execution
# Agent: Fraud Analyst
## Task: Analyze the following transactions for anomalies: ['TXN1 | Withdrawal | $500 | ATM | New York | 2024-07-01 14:23 | Card Ending: 1234', 'TXN2 | Online Purchase | $200 | Electronics Store | San Francisco | 2024-07-02 09:45 | IP: 192.168.0.1', 'TXN3 | Transfer | $1,000 | Bank Account | Zurich | 2024-07-02 17:10 | Beneficiary: John Doe', 'TXN4 | POS Purchase | $50 | Grocery Store | Berlin | 2024-07-03 12:30 | Card Ending: 5678', 'TXN5 | Withdrawal | $400 | ATM | Dubai | 2024-07-04 08:15 | Card Ending: 1234', 'TXN6 | Online Purchase | $150 | Retail Store | Paris | 2024-07-04 20:50 | IP: 10.0.0.2', 'TXN7 | Transfer | $2,000 | Bank Account | London | 2024-07-05 11:30 | Beneficiary: Jane Smith', 'TXN8 | POS Purchase | $30 | Coffee Shop | New York | 2024-07-05 15:00 | Card Ending: 5678', 'TXN9 | Withdrawal | $200 | ATM | San Francisco | 2024-07-06 10:00 | Card Ending: 1234', 'TXN10 | Online Purchase | $30